## Neural Network 

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
!pwd

/c/Users/602770/teds/PROJECT-5-CAPSTONE


In [3]:
valid_pima = pd.read_csv("../enem2017/DADOS/final_pima.csv", encoding="latin1")

In [4]:
valid_pima.head()

,student_id_num,city_of_residence,state_of_residence,age,sex,marital_status,race,nationality,birth_city_name,birth_state_initials,...,family_monthly_income,domestic_worker?,home_has_bathroom?,home_wash_machine?,home_computer,home_has_internet?,income_avr,per_house_member_income,total_score,combined_score
0,170003336360,Teresina,Piauí,30.0,M,Married,Mestizo,brazilian,Teresina,PI,...,18740+,Yes,Yes,Yes,3,Yes,18740.0,6246.666667,3558.9,2778.9
1,170001671264,Balsas,Maranhão,25.0,M,Single,Mestizo,brazilian,Balsas,MA,...,937-1406,No,Yes,Yes,1,Yes,1171.5,390.500000,2438.5,1878.5
2,170001663653,São Luís,Maranhão,23.0,F,Single,White,brazilian,São Luís,MA,...,937-1406,No,Yes,No,1,Yes,1171.5,292.875000,2884.0,2104.0
3,170001670032,Teresina,Piauí,34.0,F,Single,Mestizo,brazilian,Teresina,PI,...,0-937,No,Yes,No,0,No,468.5,156.166667,2545.8,2025.8
4,170003333636,São Luís,Maranhão,19.0,F,Single,Mestizo,brazilian,São Luís,MA,...,0-937,No,Yes,Yes,0,Yes,468.5,117.125000,2305.3,1825.3


In [5]:
valid_pima["combined_score_level"] = "Unfilled"

In [6]:
#Another new option with TensorFlow:
#combined_score_level_bucket = tf.feature_column.bucketized_column(combined_score, boundaries=[0,1600,1975,2250,2550,3300])

In [7]:
valid_pima.loc[(valid_pima["combined_score"]>= 0) & (valid_pima["combined_score"] <1600), "combined_score_level"] = "Poor"
valid_pima.loc[(valid_pima["combined_score"]>= 1600) & (valid_pima["combined_score"] <1975), "combined_score_level"] = "Fair"
valid_pima.loc[(valid_pima["combined_score"]>= 1975) & (valid_pima["combined_score"] <2250), "combined_score_level"] = "Good"
valid_pima.loc[(valid_pima["combined_score"]>= 2250) & (valid_pima["combined_score"] <2550), "combined_score_level"] = "Excellent"
valid_pima.loc[(valid_pima["combined_score"]>= 2550) & (valid_pima["combined_score"] <3300), "combined_score_level"] = "Superior"

In [8]:
valid_pima["combined_score_level"] = valid_pima["combined_score_level"].map({"Poor":0, "Fair":1, "Good":2,
                "Excellent":3, "Superior":4})

In [9]:
valid_pima.columns

Index(['student_id_num', 'city_of_residence', 'state_of_residence', 'age',
       'sex', 'marital_status', 'race', 'nationality', 'birth_city_name',
       'birth_state_initials', 'student_hs_grad_status',
       'high_school_graduation_year', 'hs_type', 'for_fun?', 'pregnant?',
       'requested_resource?', 'transgender?', 'did_ns_math?', 'did_hs_lang?',
       'ns_score', 'hs_score', 'lc_score', 'math_score', 'for_language_choice',
       'essay_gradable?', 'essay_score', 'dad_male_guardian_education',
       'mom_female_guardian_education', 'male_occupation_class',
       'female_occupation_class', 'number_of_housemates_include_student',
       'family_monthly_income', 'domestic_worker?', 'home_has_bathroom?',
       'home_wash_machine?', 'home_computer', 'home_has_internet?',
       'income_avr', 'per_house_member_income', 'total_score',
       'combined_score', 'combined_score_level'],
      dtype='object')

In [10]:
cols_to_norm = ['age', 'ns_score', 'hs_score', 'lc_score', 'math_score','income_avr', 'per_house_member_income']

In [11]:
valid_pima[cols_to_norm] = valid_pima[cols_to_norm].apply(lambda x: (x - x.min())/ (x.max() - x.min()))

#Normalizing numeric columns - not sure why 
#This is feature scaling - makes all values between 0 and 1 

In [12]:
pd.set_option("max_columns", 100)
valid_pima.head()

,student_id_num,city_of_residence,state_of_residence,age,sex,marital_status,race,nationality,birth_city_name,birth_state_initials,student_hs_grad_status,high_school_graduation_year,hs_type,for_fun?,pregnant?,requested_resource?,transgender?,did_ns_math?,did_hs_lang?,ns_score,hs_score,lc_score,math_score,for_language_choice,essay_gradable?,essay_score,dad_male_guardian_education,mom_female_guardian_education,male_occupation_class,female_occupation_class,number_of_housemates_include_student,family_monthly_income,domestic_worker?,home_has_bathroom?,home_wash_machine?,home_computer,home_has_internet?,income_avr,per_house_member_income,total_score,combined_score,combined_score_level
0,170003336360,Teresina,Piauí,0.379747,M,Married,Mestizo,brazilian,Teresina,PI,finished,before_2007,no_response,"no,_student",0,0,0,Present,Present,0.696118,0.701281,0.734052,0.713949,English,all_good,780.0,college,high_school,skilled_white5,low_white4,3.0,18740+,Yes,Yes,Yes,3,Yes,1.000000,0.333333,3558.9,2778.9,4
1,170001671264,Balsas,Maranhão,0.316456,M,Single,Mestizo,brazilian,Balsas,MA,finished,2010,no_response,"no,_student",0,0,0,Present,Present,0.324030,0.529115,0.160602,0.190357,English,all_good,560.0,completed_9th,high_school,skilled_blue3,skilled_blue3,3.0,937-1406,No,Yes,Yes,1,Yes,0.062513,0.020838,2438.5,1878.5,1
2,170001663653,São Luís,Maranhão,0.291139,F,Single,White,brazilian,São Luís,MA,finished,2012,no_response,"no,_student",0,0,0,Present,Present,0.244732,0.598991,0.561321,0.269979,Spanish,all_good,780.0,fifth_grade,fifth_grade,skilled_blue3,low_white4,4.0,937-1406,No,Yes,No,1,Yes,0.062513,0.015628,2884.0,2104.0,2
3,170001670032,Teresina,Piauí,0.430380,F,Single,Mestizo,brazilian,Teresina,PI,finished,2016,no_response,"no,_student",0,0,0,Present,Present,0.433641,0.321623,0.481132,0.267325,Spanish,all_good,520.0,fifth_grade,no_education,farmer1,farmer1,3.0,0-937,No,Yes,No,0,No,0.025000,0.008333,2545.8,2025.8,2
4,170003333636,São Luís,Maranhão,0.240506,F,Single,Mestizo,brazilian,São Luís,MA,finished,2015,no_response,"no,_student",0,0,0,Present,Present,0.211091,0.432065,0.442498,0.090681,Spanish,all_good,480.0,high_school,high_school,low_white4,unskilled_blue2,4.0,0-937,No,Yes,Yes,0,Yes,0.025000,0.006250,2305.3,1825.3,1


In [13]:
valid_pima.columns

Index(['student_id_num', 'city_of_residence', 'state_of_residence', 'age',
       'sex', 'marital_status', 'race', 'nationality', 'birth_city_name',
       'birth_state_initials', 'student_hs_grad_status',
       'high_school_graduation_year', 'hs_type', 'for_fun?', 'pregnant?',
       'requested_resource?', 'transgender?', 'did_ns_math?', 'did_hs_lang?',
       'ns_score', 'hs_score', 'lc_score', 'math_score', 'for_language_choice',
       'essay_gradable?', 'essay_score', 'dad_male_guardian_education',
       'mom_female_guardian_education', 'male_occupation_class',
       'female_occupation_class', 'number_of_housemates_include_student',
       'family_monthly_income', 'domestic_worker?', 'home_has_bathroom?',
       'home_wash_machine?', 'home_computer', 'home_has_internet?',
       'income_avr', 'per_house_member_income', 'total_score',
       'combined_score', 'combined_score_level'],
      dtype='object')

In [14]:
valid_pima.drop(['student_id_num', 'nationality', 'birth_city_name', 'birth_state_initials', 'student_hs_grad_status',
                'high_school_graduation_year', 'hs_type', 'for_fun?', 'pregnant?', 'requested_resource?', 'transgender?',
                 'did_ns_math?', 'did_hs_lang?', 'for_language_choice','essay_gradable?', 'essay_score', 
                 'mom_female_guardian_education', 'male_occupation_class','female_occupation_class', 
                 'number_of_housemates_include_student','family_monthly_income', 'domestic_worker?', 'home_has_bathroom?',
                 'home_wash_machine?', 'home_computer', 'home_has_internet?', 'total_score', 'combined_score'], axis=1, inplace=True)

In [16]:
age1 = tf.feature_column.numeric_column('age')
ns_score1 = tf.feature_column.numeric_column('ns_score')
hs_score1 = tf.feature_column.numeric_column('hs_score')
lc_score1 = tf.feature_column.numeric_column('lc_score')
math_score1 = tf.feature_column.numeric_column('math_score')
income_avr1 = tf.feature_column.numeric_column('income_avr')
per_house_member_income1 = tf.feature_column.numeric_column('per_house_member_income')             

In [17]:
state1 = tf.feature_column.categorical_column_with_vocabulary_list("state_of_residence", ["Piauí", "Maranhão"])
city1 = tf.feature_column.categorical_column_with_hash_bucket("city_of_residence", hash_bucket_size=450)
sex1 = tf.feature_column.categorical_column_with_vocabulary_list("sex", ["M", "F"])
marital_status1 = tf.feature_column.categorical_column_with_vocabulary_list("marital_status", ["Single", "Married", "Missing", "Divorced", "Widow(er)"])
race1 = tf.feature_column.categorical_column_with_vocabulary_list("race", ["Mestizo", "White", "Black", "Asian", "no_response", "Native"])
dad_educ1 = tf.feature_column.categorical_column_with_hash_bucket("dad_male_guardian_education", hash_bucket_size=10)

In [20]:
feat_cols = [age1, ns_score1, hs_score1, lc_score1, math_score1, income_avr1, per_house_member_income1, 
            state1, city1, sex1, marital_status1, race1, dad_educ1]

In [22]:
x_data = valid_pima.drop("combined_score_level", axis=1)

In [23]:
label = valid_pima["combined_score_level"]

In [25]:
#TTS
X_train, X_test, y_train, y_test = train_test_split(x_data, label, test_size=.3, random_state=12345)

In [32]:
input_function = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=5, num_epochs=1000, shuffle=True)

In [33]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols, n_classes=5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\602770\\AppData\\Local\\Temp\\tmpx89zjpo3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F9A70FC978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
model.train(input_fn=input_function, steps=100000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\602770\AppData\Local\Temp\tmpx89zjpo3\model.ckpt-13000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13000 into C:\Users\602770\AppData\Local\Temp\tmpx89zjpo3\model.ckpt.
INFO:tensorflow:loss = 3.1592395, step = 13001
INFO:tensorflow:global_step/sec: 285.367
INFO:tensorflow:loss = 4.5108814, step = 13101 (0.351 sec)
INFO:tensorflow:global_step/sec: 613.761
INFO:tensorflow:loss = 2.047579, step = 13201 (0.163 sec)
INFO:tensorflow:global_step/sec: 531.926
INFO:tensorflow:loss = 3.037497, step = 13301 (0.188 sec)
INFO:tensorflow:global_step/sec: 602.632
INFO:tensorflow:loss = 1.4449214, step = 13401 (0.167 sec)
INFO:tensorflow:global_step/sec: 645.539
INFO:tensorflow:loss = 1.2751446, step = 13501 (0.154 sec)
INF

INFO:tensorflow:loss = 1.3408308, step = 20801 (0.150 sec)
INFO:tensorflow:global_step/sec: 571.547
INFO:tensorflow:loss = 1.5044397, step = 20901 (0.172 sec)
INFO:tensorflow:global_step/sec: 695.031
INFO:tensorflow:loss = 1.7508433, step = 21001 (0.148 sec)
INFO:tensorflow:global_step/sec: 633.373
INFO:tensorflow:loss = 3.5519712, step = 21101 (0.157 sec)
INFO:tensorflow:global_step/sec: 647.499
INFO:tensorflow:loss = 2.1226676, step = 21201 (0.154 sec)
INFO:tensorflow:global_step/sec: 647.771
INFO:tensorflow:loss = 4.303469, step = 21301 (0.152 sec)
INFO:tensorflow:global_step/sec: 697.449
INFO:tensorflow:loss = 2.5530891, step = 21401 (0.144 sec)
INFO:tensorflow:global_step/sec: 606.182
INFO:tensorflow:loss = 2.1649687, step = 21501 (0.166 sec)
INFO:tensorflow:global_step/sec: 699.766
INFO:tensorflow:loss = 1.3124657, step = 21601 (0.143 sec)
INFO:tensorflow:global_step/sec: 660.361
INFO:tensorflow:loss = 1.080404, step = 21701 (0.150 sec)
INFO:tensorflow:global_step/sec: 637.288
IN

INFO:tensorflow:global_step/sec: 606.301
INFO:tensorflow:loss = 3.5153399, step = 29101 (0.162 sec)
INFO:tensorflow:global_step/sec: 680.792
INFO:tensorflow:loss = 3.3330047, step = 29201 (0.148 sec)
INFO:tensorflow:global_step/sec: 658.459
INFO:tensorflow:loss = 2.261726, step = 29301 (0.151 sec)
INFO:tensorflow:global_step/sec: 669.216
INFO:tensorflow:loss = 1.4522065, step = 29401 (0.151 sec)
INFO:tensorflow:global_step/sec: 678.488
INFO:tensorflow:loss = 1.8614469, step = 29501 (0.146 sec)
INFO:tensorflow:global_step/sec: 687.821
INFO:tensorflow:loss = 1.1680869, step = 29601 (0.146 sec)
INFO:tensorflow:global_step/sec: 619.515
INFO:tensorflow:loss = 1.7783877, step = 29701 (0.158 sec)
INFO:tensorflow:global_step/sec: 662.592
INFO:tensorflow:loss = 1.55226, step = 29801 (0.153 sec)
INFO:tensorflow:global_step/sec: 683.345
INFO:tensorflow:loss = 1.7806556, step = 29901 (0.144 sec)
INFO:tensorflow:global_step/sec: 647.635
INFO:tensorflow:loss = 2.36288, step = 30001 (0.156 sec)
INFO:

INFO:tensorflow:loss = 2.4157176, step = 37301 (0.142 sec)
INFO:tensorflow:global_step/sec: 728.168
INFO:tensorflow:loss = 5.02306, step = 37401 (0.136 sec)
INFO:tensorflow:global_step/sec: 690.208
INFO:tensorflow:loss = 1.0057051, step = 37501 (0.145 sec)
INFO:tensorflow:global_step/sec: 572.205
INFO:tensorflow:loss = 1.8303447, step = 37601 (0.176 sec)
INFO:tensorflow:global_step/sec: 554.079
INFO:tensorflow:loss = 5.303043, step = 37701 (0.183 sec)
INFO:tensorflow:global_step/sec: 602.519
INFO:tensorflow:loss = 1.086414, step = 37801 (0.165 sec)
INFO:tensorflow:global_step/sec: 664.917
INFO:tensorflow:loss = 1.3962995, step = 37901 (0.148 sec)
INFO:tensorflow:global_step/sec: 695.177
INFO:tensorflow:loss = 1.3350173, step = 38001 (0.147 sec)
INFO:tensorflow:global_step/sec: 503.716
INFO:tensorflow:loss = 1.9152472, step = 38101 (0.198 sec)
INFO:tensorflow:global_step/sec: 669.369
INFO:tensorflow:loss = 1.2404218, step = 38201 (0.149 sec)
INFO:tensorflow:global_step/sec: 668.369
INFO

INFO:tensorflow:global_step/sec: 526.246
INFO:tensorflow:loss = 2.3509243, step = 45601 (0.195 sec)
INFO:tensorflow:global_step/sec: 633.204
INFO:tensorflow:loss = 1.5817423, step = 45701 (0.153 sec)
INFO:tensorflow:global_step/sec: 673.707
INFO:tensorflow:loss = 0.863952, step = 45801 (0.150 sec)
INFO:tensorflow:global_step/sec: 649.237
INFO:tensorflow:loss = 1.0331671, step = 45901 (0.154 sec)
INFO:tensorflow:global_step/sec: 680.787
INFO:tensorflow:loss = 4.936464, step = 46001 (0.145 sec)
INFO:tensorflow:global_step/sec: 658.469
INFO:tensorflow:loss = 2.2043324, step = 46101 (0.154 sec)
INFO:tensorflow:global_step/sec: 633.121
INFO:tensorflow:loss = 1.5113995, step = 46201 (0.155 sec)
INFO:tensorflow:global_step/sec: 560.097
INFO:tensorflow:loss = 1.3029042, step = 46301 (0.180 sec)
INFO:tensorflow:global_step/sec: 590.272
INFO:tensorflow:loss = 3.50787, step = 46401 (0.167 sec)
INFO:tensorflow:global_step/sec: 632.74
INFO:tensorflow:loss = 1.4339862, step = 46501 (0.160 sec)
INFO:

...

INFO:tensorflow:global_step/sec: 469.291
INFO:tensorflow:loss = 0.8942782, step = 78601 (0.209 sec)
INFO:tensorflow:global_step/sec: 478.313
INFO:tensorflow:loss = 0.87958694, step = 78701 (0.210 sec)
INFO:tensorflow:global_step/sec: 530.508
INFO:tensorflow:loss = 0.8277399, step = 78801 (0.186 sec)
INFO:tensorflow:global_step/sec: 481.764
INFO:tensorflow:loss = 2.2912836, step = 78901 (0.213 sec)
INFO:tensorflow:global_step/sec: 472.547
INFO:tensorflow:loss = 2.156883, step = 79001 (0.208 sec)
INFO:tensorflow:global_step/sec: 468.215
INFO:tensorflow:loss = 1.6265991, step = 79101 (0.215 sec)
INFO:tensorflow:global_step/sec: 382.123
INFO:tensorflow:loss = 2.116586, step = 79201 (0.263 sec)
INFO:tensorflow:global_step/sec: 388.039
INFO:tensorflow:loss = 1.4719218, step = 79301 (0.256 sec)
INFO:tensorflow:global_step/sec: 497.436
INFO:tensorflow:loss = 2.1587882, step = 79401 (0.199 sec)
INFO:tensorflow:global_step/sec: 380.548
INFO:tensorflow:loss = 0.85497856, step = 79501 (0.264 sec)


INFO:tensorflow:loss = 1.0792093, step = 86801 (0.162 sec)
INFO:tensorflow:global_step/sec: 645.437
INFO:tensorflow:loss = 1.3640162, step = 86901 (0.155 sec)
INFO:tensorflow:global_step/sec: 613.759
INFO:tensorflow:loss = 1.1952337, step = 87001 (0.164 sec)
INFO:tensorflow:global_step/sec: 382.859
INFO:tensorflow:loss = 1.2956631, step = 87101 (0.264 sec)
INFO:tensorflow:global_step/sec: 507.516
INFO:tensorflow:loss = 2.2526624, step = 87201 (0.195 sec)
INFO:tensorflow:global_step/sec: 546.497
INFO:tensorflow:loss = 1.9559512, step = 87301 (0.182 sec)
INFO:tensorflow:global_step/sec: 519.403
INFO:tensorflow:loss = 1.0611323, step = 87401 (0.193 sec)
INFO:tensorflow:global_step/sec: 473.739
INFO:tensorflow:loss = 3.576538, step = 87501 (0.212 sec)
INFO:tensorflow:global_step/sec: 479.315
INFO:tensorflow:loss = 0.52946997, step = 87601 (0.207 sec)
INFO:tensorflow:global_step/sec: 416.472
INFO:tensorflow:loss = 1.7688003, step = 87701 (0.238 sec)
INFO:tensorflow:global_step/sec: 474.932


INFO:tensorflow:loss = 1.8897984, step = 95001 (0.174 sec)
INFO:tensorflow:global_step/sec: 478.234
INFO:tensorflow:loss = 1.1049516, step = 95101 (0.206 sec)
INFO:tensorflow:global_step/sec: 537.659
INFO:tensorflow:loss = 1.654385, step = 95201 (0.188 sec)
INFO:tensorflow:global_step/sec: 549.509
INFO:tensorflow:loss = 1.9063457, step = 95301 (0.178 sec)
INFO:tensorflow:global_step/sec: 557.183
INFO:tensorflow:loss = 1.5211887, step = 95401 (0.179 sec)
INFO:tensorflow:global_step/sec: 515.431
INFO:tensorflow:loss = 1.6206174, step = 95501 (0.195 sec)
INFO:tensorflow:global_step/sec: 588.412
INFO:tensorflow:loss = 1.004684, step = 95601 (0.168 sec)
INFO:tensorflow:global_step/sec: 566.311
INFO:tensorflow:loss = 3.1089468, step = 95701 (0.179 sec)
INFO:tensorflow:global_step/sec: 595.467
INFO:tensorflow:loss = 1.690947, step = 95801 (0.166 sec)
INFO:tensorflow:global_step/sec: 656.194
INFO:tensorflow:loss = 1.0997648, step = 95901 (0.153 sec)
INFO:tensorflow:global_step/sec: 558.695
INF

INFO:tensorflow:loss = 1.1919563, step = 103201 (0.143 sec)
INFO:tensorflow:global_step/sec: 652.325
INFO:tensorflow:loss = 1.6078253, step = 103301 (0.152 sec)
INFO:tensorflow:global_step/sec: 610.004
INFO:tensorflow:loss = 1.5022963, step = 103401 (0.165 sec)
INFO:tensorflow:global_step/sec: 641.391
INFO:tensorflow:loss = 1.3639947, step = 103501 (0.155 sec)
INFO:tensorflow:global_step/sec: 604.45
INFO:tensorflow:loss = 1.7831619, step = 103601 (0.168 sec)
INFO:tensorflow:global_step/sec: 727.832
INFO:tensorflow:loss = 2.7742002, step = 103701 (0.136 sec)
INFO:tensorflow:global_step/sec: 635.276
INFO:tensorflow:loss = 0.9710974, step = 103801 (0.156 sec)
INFO:tensorflow:global_step/sec: 674.026
INFO:tensorflow:loss = 1.5736119, step = 103901 (0.150 sec)
INFO:tensorflow:global_step/sec: 658.181
INFO:tensorflow:loss = 0.8850462, step = 104001 (0.153 sec)
INFO:tensorflow:global_step/sec: 631.047
INFO:tensorflow:loss = 1.709111, step = 104101 (0.162 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:global_step/sec: 519.462
INFO:tensorflow:loss = 1.347141, step = 111401 (0.191 sec)
INFO:tensorflow:global_step/sec: 407.919
INFO:tensorflow:loss = 1.0674123, step = 111501 (0.247 sec)
INFO:tensorflow:global_step/sec: 466.053
INFO:tensorflow:loss = 0.8059323, step = 111601 (0.218 sec)
INFO:tensorflow:global_step/sec: 449.264
INFO:tensorflow:loss = 2.8611753, step = 111701 (0.219 sec)
INFO:tensorflow:global_step/sec: 529.055
INFO:tensorflow:loss = 1.05215, step = 111801 (0.194 sec)
INFO:tensorflow:global_step/sec: 560.366
INFO:tensorflow:loss = 1.5476744, step = 111901 (0.174 sec)
INFO:tensorflow:global_step/sec: 543.447
INFO:tensorflow:loss = 2.8384688, step = 112001 (0.186 sec)
INFO:tensorflow:global_step/sec: 519.307
INFO:tensorflow:loss = 2.669564, step = 112101 (0.191 sec)
INFO:tensorflow:global_step/sec: 584.963
INFO:tensorflow:loss = 2.259729, step = 112201 (0.172 sec)
INFO:tensorflow:global_step/sec: 536.214
INFO:tensorflow:loss = 1.2248714, step = 112301 (0.185 

In [48]:
eval_input_function = tf.estimator.inputs.pandas_input_fn(x=X_test, y=y_test, batch_size=10, num_epochs=1, shuffle=False)

In [49]:
results = model.evaluate(eval_input_function)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-05T21:22:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\602770\AppData\Local\Temp\tmpx89zjpo3\model.ckpt-113000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-05-21:22:29
INFO:tensorflow:Saving dict for global step 113000: accuracy = 0.91366196, average_loss = 0.30494237, global_step = 113000, loss = 3.0491867
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 113000: C:\Users\602770\AppData\Local\Temp\tmpx89zjpo3\model.ckpt-113000


In [50]:
results

{'accuracy': 0.91366196,
 'average_loss': 0.30494237,
 'loss': 3.0491867,
 'global_step': 113000}

## Dense Neural Network  - DNN Classifier 